Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Spritemap generation -- Building Blocks of Interpretability

This colab notebook is part of our **Building Blocks of Intepretability** series exploring how intepretability techniques combine together to explain neural networks. If you haven't already, make sure to look at the [**corresponding paper**](https://distill.pub/2018/building-blocks) as well!

This notebook allow to generate spritemaps used in the notebooks *Semantic dictionnary* and *Channel attribution*. This notebook is meant to be used with Jupyter lab or Jupyter notebook only.

In [2]:
from math import ceil

import numpy as np

import tensorflow as tf
# uncomment to avoid deprecation warnings :
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
tf.logging.set_verbosity(tf.compat.v1.logging.ERROR)

import lucid.modelzoo.vision_models as models
from lucid.optvis import render, objectives, transform, param
from lucid.misc.channel_reducer import ChannelReducer
from lucid.misc.io import show, load, save

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.decomposition.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
model = models.InceptionV1()
model.layers

(Layer (belonging to InceptionV1) <conv2d0: 64> ([{'conv'}]),
 Layer (belonging to InceptionV1) <conv2d1: 64> ([{'conv'}]),
 Layer (belonging to InceptionV1) <conv2d2: 192> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed3a: 256> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed3b: 480> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4a: 508> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4b: 512> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4c: 512> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4d: 528> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed4e: 832> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed5a: 832> ([{'conv'}]),
 Layer (belonging to InceptionV1) <mixed5b: 1024> ([{'conv'}]),
 Layer (belonging to InceptionV1) <head0_bottleneck: 128> ([{'conv'}]),
 Layer (belonging to InceptionV1) <nn0: 1024> ([{'dense'}]),
 Layer (belonging to InceptionV1) <softmax0: 1008> ([{'dense'}]),
 Layer (belonging to InceptionV1) <head1_bottl

### Layer 6 - Mixed 4d

In [4]:
def create_spritemap(model, layer_name, sprite_size, row_size, column_size=24):
    param_funct = lambda : param.image(sprite_size)
        
    height = column_size * sprite_size
    width = row_size * sprite_size
    
    neuron_sprite_map_array = np.full((height,width,3), 0.9, dtype="single")
    channel_sprite_map_array = np.full((height,width,3), 0.9, dtype="single")
    
    for i in range(column_size):
        top_left_y = sprite_size * i
        for j in range(row_size):
            top_left_x = sprite_size * j
            unit_index = i*row_size+j
            
            neuron_objectif_f = objectives.neuron(layer_name, channel_n=unit_index)
            neuron_feature_viz = render.render_vis(model, neuron_objectif_f, param_funct, verbose=False)
            neuron_sprite_map_array[top_left_y : top_left_y+sprite_size,
                             top_left_x : top_left_x+sprite_size,:] = neuron_feature_viz[0][0]
            
            channel_objectif_f = layer_name + ":" + str(unit_index) 
            channel_feature_viz = render.render_vis(model, channel_objectif_f, param_funct, verbose=False)
            channel_sprite_map_array[top_left_y : top_left_y+sprite_size,
                             top_left_x : top_left_x+sprite_size,:] = channel_feature_viz[0][0]
    
    return neuron_sprite_map_array, channel_sprite_map_array

In [7]:
column_size = 24
sprite_size = 110

for layer in model.layers[3:12]:
    print(layer.name, layer.depth)
    row_size = ceil(model.get_layer(layer.name).depth / column_size)
    
    neuron_sprite_map_array, channel_sprite_map_array = create_spritemap(model, layer.name, sprite_size, row_size, column_size)
    
    save(neuron_sprite_map_array, "./spritemaps/spritemap_neuron_" + layer.name.split("/")[0] + ".jpeg")
    save(channel_sprite_map_array, "./spritemaps/spritemap_channel_" + layer.name.split("/")[0] + ".jpeg")
    
    # we use split("/") to avoid  path problems with layer names containing "/"

mixed3a 256


NameError: name 'neuron_sprite_map_nam' is not defined